In [2]:
import Pkg
Pkg.add("Clustering")

   Resolving package versions...
   Installed NearestNeighbors ─ v0.4.16
   Installed Clustering ─────── v0.15.7
    Updating `C:\Users\matsz\.julia\environments\v1.10\Project.toml`
  [aaaa29a8] + Clustering v0.15.7
    Updating `C:\Users\matsz\.julia\environments\v1.10\Manifest.toml`
  [aaaa29a8] + Clustering v0.15.7
  [b8a86587] + NearestNeighbors v0.4.16
Precompiling project...
  ✓ NearestNeighbors
  ✓ Clustering
  2 dependencies successfully precompiled in 24 seconds. 395 already precompiled.


In [343]:
using Clustering
using CSV
using DataFrames
using Statistics
using StatsBase
using TimeSeries
using HiddenMarkovModels


In [273]:
acwi = Matrix(CSV.read("../sb_gbi/equity_scenarios.csv",DataFrame))

99999×50 Matrix{Float64}:
  0.183608   0.040727   0.130638  …   0.310636   0.259417   0.106576
  0.057796  -0.015669   0.01473       0.181475   0.038817   0.09112
 -0.042034   0.071062   0.211914      0.271774   0.03051    0.014144
  0.048233   0.002585   0.069304      0.161997   0.150916  -0.458586
 -0.097277   0.278737   0.249458      0.054621   0.20249    0.148626
  0.168582   0.058298   0.174671  …  -0.057467  -0.037438   0.030723
  0.096997   0.214256  -0.246285      0.148997   0.192833   0.181174
  0.254872   0.054849   0.014113      0.13077    0.066608  -0.076274
  0.263792   0.213822   0.260164      0.309639   0.022386   0.268776
  0.065249  -0.206639   0.347267      0.165357   0.180862   0.298468
  ⋮                               ⋱                        
  0.304079  -0.004624   0.041851  …  -0.008278   0.136057   0.139732
  0.261117   0.094383  -0.240846      0.037133  -0.072158  -0.037526
 -0.050816   0.211829  -0.100711      0.158417  -0.021944   0.318735
  0.370563   0.279

In [274]:
inflation = Matrix(CSV.read("../sb_gbi/inflation_scenarios.csv", DataFrame))[:,2:end]

99999×50 Matrix{Float64}:
 0.035158  0.031416  0.046096  0.018379  …  0.06628   0.09403   0.093851
 0.013222  0.0123    0.043899  0.027086     0.019199  0.021789  0.020151
 0.039912  0.040263  0.035394  0.0123       0.02551   0.0123    0.0123
 0.016048  0.0123    0.0123    0.03366      0.056582  0.0123    0.0123
 0.065174  0.052422  0.071214  0.05068      0.019615  0.058837  0.048566
 0.051986  0.035766  0.026192  0.033541  …  0.052188  0.046143  0.026802
 0.040925  0.025968  0.043467  0.024486     0.0786    0.047548  0.027347
 0.0123    0.045685  0.022954  0.0123       0.019471  0.048676  0.043244
 0.0123    0.014093  0.0123    0.0123       0.037708  0.071717  0.083529
 0.0123    0.046767  0.048754  0.071964     0.058765  0.0123    0.024597
 ⋮                                       ⋱                      
 0.05639   0.034089  0.038381  0.037493  …  0.037054  0.031733  0.02699
 0.022314  0.042175  0.031607  0.039552     0.060712  0.045391  0.080075
 0.078099  0.090769  0.052886  0.02624

In [275]:
returns = stack([acwi,inflation],dims=1)

2×99999×50 Array{Float64, 3}:
[:, :, 1] =
 0.183608  0.057796  -0.042034  0.048233  …  0.119115  -0.145221  -0.025328
 0.035158  0.013222   0.039912  0.016048     0.064728   0.027103   0.0123

[:, :, 2] =
 0.040727  -0.015669  0.071062  0.002585  …  0.095772  -0.393472  0.014187
 0.031416   0.0123    0.040263  0.0123       0.058533   0.059451  0.027505

[:, :, 3] =
 0.130638  0.01473   0.211914  0.069304  …  -0.292217  0.060243  0.174296
 0.046096  0.043899  0.035394  0.0123        0.036324  0.051567  0.0123

;;; … 

[:, :, 48] =
 0.310636  0.181475  0.271774  0.161997  …  0.084165  0.142273  0.012147
 0.06628   0.019199  0.02551   0.056582     0.0123    0.04997   0.065018

[:, :, 49] =
 0.259417  0.038817  0.03051  0.150916  …  0.118251  0.023003  0.20869
 0.09403   0.021789  0.0123   0.0123       0.035531  0.0123    0.095034

[:, :, 50] =
 0.106576  0.09112   0.014144  -0.458586  …  0.292593  0.115302  0.170179
 0.093851  0.020151  0.0123     0.0123       0.045336  0.0123    0.057752

In [276]:
period_1 = returns[:,:,1] 


2×99999 Matrix{Float64}:
 0.183608  0.057796  -0.042034  0.048233  …  0.119115  -0.145221  -0.025328
 0.035158  0.013222   0.039912  0.016048     0.064728   0.027103   0.0123

In [277]:
period_1[:,1]

2-element Vector{Float64}:
 0.183608
 0.035158

In [307]:
n_clusters = 12
result = kmeans(period_1,n_clusters)

KmeansResult{Matrix{Float64}, Float64, Int64}([0.16296098013307034 -0.2607995752833247 … -0.14868747768853233 0.10388507706971659; 0.02547086955299499 0.04010536805180805 … 0.04120343908969239 0.03718827920751529], [1, 5, 8, 5, 8, 7, 12, 4, 4, 5  …  9, 4, 4, 8, 10, 11, 11, 12, 11, 9], [0.0005201399256826861, 0.000803386017654591, 0.0006296438464753473, 0.0006404017351488928, 0.001509075640955846, 0.00047008516119219357, 6.140868799934737e-5, 0.0013509614031674555, 0.0010209900769014568, 0.000993098144555525  …  0.000625418641668531, 0.000925805572992433, 0.0006286157278189985, 0.0016648237462993046, 0.0003839288010855091, 0.000475828103214386, 0.0006211010153250734, 0.0009903867737903853, 0.00021083885008722075, 0.0012657172098888734], [10672, 1854, 1594, 9380, 13579, 12832, 7540, 7447, 10602, 5315, 4490, 14694], [10671, 1853, 1595, 9384, 13587, 12841, 7536, 7442, 10597, 5323, 4482, 14688], 82.10616191144553, 100, false)

In [308]:
counts(result)

12-element Vector{Int64}:
 10672
  1854
  1594
  9380
 13579
 12832
  7540
  7447
 10602
  5315
  4490
 14694

In [309]:
centers =result.centers

2×12 Matrix{Float64}:
 0.162961   -0.2608     0.461954   …  0.351953   -0.148687   0.103885
 0.0254709   0.0401054  0.0401165     0.0400039   0.0412034  0.0371883

In [310]:
for cluster in 1:n_clusters
    println("equity: $(centers[1,cluster]), infl: $(centers[2,cluster])")
end

equity: 0.16296098013307034, infl: 0.02547086955299499
equity: -0.2607995752833247, infl: 0.04010536805180805
equity: 0.46195439059561116, infl: 0.04011652413793122
equity: 0.27782815057544835, infl: 0.04100499179454338
equity: 0.05229195510414353, infl: 0.04102652314712459
equity: 0.2178176394361799, infl: 0.04114076356981431
equity: 0.15181469413481977, infl: 0.06573163986199561
equity: -0.06711453654931478, infl: 0.0406933660306371
equity: -0.003215540058507103, infl: 0.040170348494856414
equity: 0.35195310182228, infl: 0.04000392391508608
equity: -0.14868747768853233, infl: 0.04120343908969239
equity: 0.10388507706971659, infl: 0.03718827920751529


In [311]:
clusters = assignments(result)

99999-element Vector{Int64}:
  1
  5
  8
  5
  8
  7
 12
  4
  4
  5
  ⋮
  4
  4
  8
 10
 11
 11
 12
 11
  9

In [312]:
cluster_means = zeros((2,n_clusters))

for cluster in 1:n_clusters
    cluster_means[:,cluster] = mean(period_1[:,clusters .== cluster],dims=2) 
end


for cluster in 1:n_clusters
    println("$(cluster_means[1,cluster]), $(cluster_means[2,cluster])")
end

0.16307093787481203, 0.025488097170165792
-0.260769216289105, 0.040090370550161986
0.46198891718946034, 0.04011359598494373
0.2779050748400861, 0.04097765309168392
0.052351660063332965, 0.04102963347816347
0.21789597778989925, 0.04115732091645774
0.15181621816976146, 0.06571632904509268
-0.0670053844501142, 0.04067359795890979
-0.0031234698170156377, 0.04017446161101617
0.35202665776105296, 0.04003211194731951
-0.14859332093541247, 0.04121935768374194
0.10393540411052112, 0.0371736597250568


In [313]:
probs = counts(result)/sum(counts(result))

12-element Vector{Float64}:
 0.10672106721067211
 0.01854018540185402
 0.015940159401594017
 0.0938009380093801
 0.13579135791357913
 0.12832128321283212
 0.07540075400754008
 0.07447074470744708
 0.1060210602106021
 0.053150531505315055
 0.04490044900449004
 0.14694146941469416

In [314]:
scenario_period1 = DataFrame(Equity=cluster_means[1,:].*100,EDO=cluster_means[2,:].*100,Probs=probs*100)

Row,Equity,EDO,Probs
,Float64,Float64,Float64
1,16.3071,2.54881,10.6721
2,-26.0769,4.00904,1.85402
3,46.1989,4.01136,1.59402
4,27.7905,4.09777,9.38009
5,5.23517,4.10296,13.5791
6,21.7896,4.11573,12.8321
7,15.1816,6.57163,7.54008
8,-6.70054,4.06736,7.44707
9,-0.312347,4.01745,10.6021


In [324]:
period_1

2×99999 Matrix{Float64}:
 0.183608  0.057796  -0.042034  0.048233  …  0.119115  -0.145221  -0.025328
 0.035158  0.013222   0.039912  0.016048     0.064728   0.027103   0.0123

In [331]:
function cluster(data, n_clusters)
    dt = fit(ZScoreTransform, data, dims=2)
    standarized = StatsBase.transform(dt,data)

    clusters = kmeans(standarized,n_clusters)

    destandarised = StatsBase.reconstruct(dt, clusters.centers)

    probs = counts(clusters)/sum(counts(clusters))

    return destandarised, probs
end

grid, probs = cluster(period_1, 6)
display(DataFrame(grid, :auto))
display(probs)

Row,x1,x2,x3,x4,x5,x6
,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0951617,0.000456613,0.283282,0.243415,-0.105125,0.108636
2,0.0180806,0.0715669,0.026562,0.0672825,0.031034,0.0450768


6-element Vector{Float64}:
 0.21789217892178922
 0.11935119351193511
 0.17094170941709416
 0.13683136831368314
 0.12454124541245412
 0.23044230442304423

In [360]:
function cluster_moments(data, n_clusters)
    dt = fit(ZScoreTransform, data, dims=2)
    standarized = StatsBase.transform(dt,data)

    clusters = kmeans(standarized,n_clusters)
    c = assignments(clusters)

    cluster_means = zeros(2,n_clusters)
    cluster_cov = zeros(2,2,n_clusters)

    for cluster in 1:n_clusters
        cluster_means[:,cluster] = mean(period_1[:,c .== cluster],dims=2) 
        cluster_cov[:,:,cluster] = cov(period_1[:,c .== cluster],dims=2) 
    end

    return  cluster_means, cluster_cov 
end

means, c_cov = cluster_moments(period_1,6)


([0.095549538055696 -0.0001921960472322236 … 0.28350519279859615 0.10882563169313489; 0.018078551039132463 0.07147965363034925 … 0.02660462154566618 0.04508088019612947], [0.0036114409415838535 -2.9988800968336833e-6; -2.9988800968336833e-6 4.138922750818814e-5;;; 0.007478499580348579 0.00015920509009783492; 0.00015920509009783492 0.00016593266471954903;;; 0.00634682104909424 -5.5400622283823644e-5; -5.5400622283823644e-5 0.0001729726331389562;;; 0.005956258029793182 -0.00014089998496450492; -0.00014089998496450492 0.000172267929365687;;; 0.0052332223456228195 5.921181675878272e-5; 5.921181675878272e-5 0.0001241432557807641;;; 0.0034121656096731553 -2.603569047684756e-6; -2.603569047684756e-6 5.85925271508015e-5])

In [351]:
function guess_init(n_scenarios)
    regimes_probs = rand(1:100,n_scenarios)
    return regimes_probs/sum(regimes_probs)
end

function guess_tmatrix(n_scenarios)
    regimes_probs = rand(1:100,n_scenarios,n_scenarios)
    return regimes_probs ./ sum(regimes_probs,dims=2)
end
function guess_distribution()
    corr = Matrix(1.0I,2,2)
    sigma = [rand(Uniform(0,0.25)),rand(Uniform(0,0.035))]
    means = [rand(Uniform(-0.5,0.5)),rand(Uniform(-0.01,0.12))]
    cov = cor2cov(corr,sigma)
    return MvNormal(means,cov)
end

using LinearAlgebra, Distributions

In [366]:
n_clusters = 6
init_guess = guess_init(n_clusters)
guess_matrix = guess_tmatrix(n_clusters)
guess_dist = [guess_distribution() for s in 1:n_clusters]
hmm_guess = HMM(init_guess, guess_matrix, guess_dist);
println(hmm_guess)
hmm_est_concat, _ = baum_welch(hmm_guess, eachcol(period_1));
println(hmm_est_concat)

Hidden Markov Model with:
 - initialization: [0.045774647887323945, 0.2676056338028169, 0.1795774647887324, 0.2640845070422535, 0.1056338028169014, 0.13732394366197184]
 - transition matrix: [0.19242902208201892 0.14195583596214512 0.09779179810725552 0.06624605678233439 0.28391167192429023 0.21766561514195584; 0.18662952646239556 0.116991643454039 0.12813370473537605 0.116991643454039 0.18662952646239556 0.2646239554317549; 0.23776223776223776 0.2902097902097902 0.1048951048951049 0.05244755244755245 0.17832167832167833 0.13636363636363635; 0.08917197452229299 0.2929936305732484 0.012738853503184714 0.1464968152866242 0.24522292993630573 0.21337579617834396; 0.06896551724137931 0.28448275862068967 0.23275862068965517 0.22413793103448276 0.10919540229885058 0.08045977011494253; 0.21739130434782608 0.2080745341614907 0.13975155279503104 0.12422360248447205 0.2204968944099379 0.09006211180124224]
 - observation distributions: [FullNormal(
dim: 2
μ: [-0.04483279065546819, 0.08710305753048

In [ ]:
Hidden Markov Model with:
 - initialization: [4.057191263733909e-58, 8.58307770403401e-34, 3.522376617531163e-30, 1.0, 2.13452492606223e-70, 0.0]
 - transition matrix: [0.08162785050933843 0.12578264175977324 0.0292083305518417 0.5999289376466184 0.028914296436569557 0.13453794309585868; 0.07083091069559361 0.11807662310172207 0.033329344227112215 0.6014035445537554 0.023189023541915453 0.1531705538799011; 0.08950521994378781 0.1449451884624156 0.05072817185668049 0.5363641169187194 0.020759734533418315 0.15769756828497847; 0.050632057044251486 0.14090289833097958 0.020370114792142634 0.6095987359969572 0.032322058284570444 0.14617413555109865; 0.027478849112433708 0.1666630725107338 0.03616403108124411 0.608385736538321 0.026653839305742257 0.13465447145152534; 0.0476826279625842 0.133153593230571 0.02777814240724316 0.6194116746645544 0.027192126144850903 0.14478183559019636]
 - observation distributions: [FullNormal(
dim: 2
μ: [0.07361955859928039, 0.06589744656778986]
Σ: [0.011583921705941509 0.0007187536324547843; 0.0007187536324547843 0.00021256925929568317]
)
, FullNormal(
dim: 2
μ: [0.14466175876378304, 0.07094996156224362]
Σ: [0.018515289752669858 -0.00021540439347833738; -0.00021540439347833738 0.00025476072065482876]
)
, FullNormal(
dim: 2
μ: [-0.08924354085063477, 0.028703954775813646]
Σ: [0.014011385733960408 6.64747219541934e-5; 6.64747219541934e-5 9.556273284176555e-5]
)
, FullNormal(
dim: 2
μ: [0.13269966732806096, 0.03782987995376803]
Σ: [0.015699360009847948 -3.125982846534618e-6; -3.125982846534618e-6 0.00021011427123105477]
)
, FullNormal(
dim: 2
μ: [-0.14668004688969205, 0.056206938421529865]
Σ: [0.009867988667763474 0.00022133753969860192; 0.00022133753969860192 0.0003603936317602492]
)
, FullNormal(
dim: 2
μ: [0.1138565867469055, 0.012300000000001688]
Σ: [0.020011302843007513 -3.0846777324899687e-31; -3.0846777324899687e-31 2.8489589536246626e-30]
)
]


In [367]:
n_clusters = 6
guess_dist = [MvNormal(means[:,s],c_cov[:,:,s]) for s in 1:n_clusters]
init_guess = guess_init(n_clusters)
guess_matrix = guess_tmatrix(n_clusters)
hmm_guess = HMM(init_guess, guess_matrix, guess_dist);
println(hmm_guess)
hmm_est_concat, _ = baum_welch(hmm_guess, eachcol(period_1));
println(hmm_est_concat)

Hidden Markov Model with:
 - initialization: [0.2664359861591695, 0.1522491349480969, 0.27335640138408307, 0.1453287197231834, 0.0657439446366782, 0.09688581314878893]
 - transition matrix: [0.20556745182012848 0.21413276231263384 0.03854389721627409 0.21199143468950749 0.16916488222698073 0.16059957173447537; 0.2182890855457227 0.17994100294985252 0.2182890855457227 0.0029498525073746312 0.2713864306784661 0.10914454277286136; 0.1925925925925926 0.1 0.32222222222222224 0.24814814814814815 0.1037037037037037 0.03333333333333333; 0.32653061224489793 0.37755102040816324 0.02040816326530612 0.10714285714285714 0.030612244897959183 0.1377551020408163; 0.23310810810810811 0.2195945945945946 0.0033783783783783786 0.28040540540540543 0.18243243243243243 0.08108108108108109; 0.10397553516819572 0.29357798165137616 0.13149847094801223 0.1926605504587156 0.11314984709480122 0.1651376146788991]
 - observation distributions: [FullNormal(
dim: 2
μ: [0.095549538055696, 0.018078551039132463]
Σ: [0.00

In [ ]:
Hidden Markov Model with:
 - initialization: [0.0, 5.506266603129145e-35, 1.2360419849940215e-26, 9.166156952817547e-19, 0.9995505578623286, 0.0004494421376713598]
 - transition matrix: [0.14478183559017818 0.18132115158123324 0.08140277363511825 0.16558371093574914 0.17357833898338898 0.25333218927433226; 0.1424134098482857 0.21502628674409785 0.06883161353411733 0.12670696009864563 0.17673077285061817 0.2702909569242352; 0.14715363640360216 0.1862043022151603 0.08829611468061177 0.19219189737859685 0.1642836517997844 0.22187039752224452; 0.1559315662157372 0.18300304675758808 0.08704150855593899 0.17541048459943048 0.15590902956014174 0.24270436431116343; 0.14729583788084735 0.20615825981819508 0.07705693764142892 0.14504419089238094 0.16419676970435812 0.2602480040627896; 0.14306945010572408 0.1933087050728161 0.0737151232117841 0.15959275412532384 0.16929110454432691 0.26102286294002486]
 - observation distributions: [FullNormal(
dim: 2
μ: [0.11385658674690693, 0.012300000000002738]
Σ: [0.020011302843007558 -1.0579975778869756e-30; -1.0579975778869756e-30 7.493323968186792e-30]
)
, FullNormal(
dim: 2
μ: [0.04519261499791618, 0.0638420995810877]
Σ: [0.022468812744550835 0.000792095303295707; 0.000792095303295707 0.0003757837847281989]
)
, FullNormal(
dim: 2
μ: [-0.031021295161304228, 0.031561522755413025]
Σ: [0.013871211997324656 0.00011142389242223676; 0.00011142389242223676 0.00011331386723773387]
)
, FullNormal(
dim: 2
μ: [0.2051301054069273, 0.056982047489017994]
Σ: [0.014993007596771824 -0.00042747569038760003; -0.00042747569038760003 0.00021248413615301416]
)
, FullNormal(
dim: 2
μ: [0.18075140914223486, 0.02488042524292236]
Σ: [0.013234172030043961 0.00013963841312327645; 0.00013963841312327645 6.045618159987285e-5]
)
, FullNormal(
dim: 2
μ: [0.11312284712633551, 0.04157544194288699]
Σ: [0.008473883117019977 0.00011477490710790708; 0.00011477490710790708 0.00016426019403165625]
)
]


In [317]:
source_data = TimeArray(CSV.File("assets_data_03.24.csv", delim=';', dateformat=Dict("Date" => "dd.mm.yyyy"), decimal=','),timestamp=:Date)
returns = percentchange(source_data, :log)


615×5 TimeArray{Union{Missing, Float64}, 2, Date, Matrix{Union{Missing, Float64}}} 1973-01-31 to 2024-03-31
┌────────────┬────────────┬─────────────┬──────────────┬──────────────┬─────────
│            │ ACWI_Net   │ USDPLN      │ ACWI_Net_PLN │ CPI_USA      │ CPI_PL ⋯
├────────────┼────────────┼─────────────┼──────────────┼──────────────┼─────────
│ 1973-01-31 │    missing │     missing │      missing │   0.00235067 │     mi ⋯
│ 1973-02-28 │    missing │     missing │      missing │   0.00701905 │     mi ⋯
│ 1973-03-31 │    missing │     missing │      missing │   0.00927727 │     mi ⋯
│ 1973-04-30 │    missing │     missing │      missing │   0.00690598 │     mi ⋯
│ 1973-05-31 │    missing │     missing │      missing │   0.00685862 │     mi ⋯
│ 1973-06-30 │    missing │     missing │      missing │    0.0068119 │     mi ⋯
│ 1973-07-31 │    missing │     missing │      missing │   0.00225501 │     mi ⋯
│ 1973-08-31 │    missing │     missing │      missing │    0.0179014 │     mi ⋯
│

In [318]:
returns_yearly = percentchange(collapse(source_data,year, last))

52×5 TimeArray{Union{Missing, Float64}, 2, Date, Matrix{Union{Missing, Float64}}} 1973-12-31 to 2024-03-31
┌────────────┬────────────┬────────────┬──────────────┬────────────┬────────────
│            │ ACWI_Net   │ USDPLN     │ ACWI_Net_PLN │ CPI_USA    │ CPI_PLN   ⋯
├────────────┼────────────┼────────────┼──────────────┼────────────┼────────────
│ 1973-12-31 │    missing │    missing │      missing │  0.0870605 │     missi ⋯
│ 1974-12-31 │    missing │    missing │      missing │   0.123377 │     missi ⋯
│ 1975-12-31 │    missing │    missing │      missing │  0.0693605 │     missi ⋯
│ 1976-12-31 │    missing │    missing │      missing │  0.0486505 │     missi ⋯
│ 1977-12-31 │    missing │    missing │      missing │  0.0670123 │     missi ⋯
│ 1978-12-31 │    missing │    missing │      missing │   0.090177 │     missi ⋯
│ 1979-12-31 │    missing │    missing │      missing │   0.132939 │     missi ⋯
│ 1980-12-31 │    missing │    missing │      missing │   0.125161 │     missi ⋯
│ 

In [319]:
val = eachcol(values(from(returns_yearly,Date(2003,02,01))))

[percentile(a,[5, 25, 50, 75, 95]) for a in val]

5-element Vector{Vector{Float64}}:
 [-0.1791646535637412, 0.050866825980199704, 0.13951720841975523, 0.21889844750843207, 0.3361762844747671]
 [-0.16789023904324368, -0.08309423208140795, 0.01405036560292702, 0.08138626868521559, 0.17291389833351883]
 [-0.11150993397204947, 0.03390535467137583, 0.09733228505343647, 0.19875932356089762, 0.30291930341482903]
 [0.007308352896474506, 0.015616012312351955, 0.02091860212971578, 0.03182293580839392, 0.06335757465785029]
 [-0.005813596241253867, 0.00972740828317448, 0.02302606076369674, 0.03842710017413295, 0.08567916010546725]

In [320]:
acwi_cpipln = identity.(values(from(returns[:ACWI_Net_PLN,:CPI_PLN],Date(2003,02,01))))

254×2 Matrix{Float64}:
  0.0051036     0.000927653
  0.0430686     0.00277912
  0.0118412     0.0018489
  0.0300994     0.0
  0.0670022    -0.000924682
  0.0132941    -0.00370334
  0.0495565    -0.00371711
  0.000195397   0.0046439
  0.0782452     0.00554541
 -0.0161735     0.00276122
  ⋮            
  0.0208096    -0.00221661
  0.000986609   0.0
  0.0154043    -0.00389282
 -0.066645      0.00278164
  0.0356998     0.00719694
  0.0310713     0.00110186
  0.0233511     0.00384933
  0.0394925     0.00274048
  0.0298368     0.00218699

In [321]:
transpose(acwi_cpipln)

2×254 transpose(::Matrix{Float64}) with eltype Float64:
 0.0051036    0.0430686   0.0118412  …  0.0233511   0.0394925   0.0298368
 0.000927653  0.00277912  0.0018489     0.00384933  0.00274048  0.00218699

In [322]:
state, probs = cluster(transpose(acwi_cpipln),4)
display(state*12)
display(probs)


2×4 Matrix{Float64}:
 0.506915   -0.741961   -0.0241622  -0.0317024
 0.0235401   0.0357811   0.0073365   0.143168

4-element Vector{Float64}:
 0.33858267716535434
 0.09055118110236221
 0.44881889763779526
 0.1220472440944882

In [323]:
acwi_cpiusd = identity.(values(from(returns[:ACWI_Net,:CPI_USA],Date(1988,01,01))))

state, probs = cluster(transpose(acwi_cpiusd),3)

display(state)
display(probs)

2×3 Matrix{Float64}:
 0.0208741    0.0300136    -0.0488946
 0.00509747  -0.000185102   0.00239701

3-element Vector{Float64}:
 0.34022988505747126
 0.39770114942528734
 0.2620689655172414